# Inteligência Competitiva

O Exercício a seguir deve ser desenvolvido utilizando o ambiente R. Deve ser entregue o código fonte, comentado, em R. O prazo final para entrega da lista é o dia 28 de novembro, no horário da aula.

Criar um script que:

a) Leia do Twitter mensagens que contenham a palavra Olimpíada a partir da data 17/09/2016 e que estejam em um raio de 60Km da latitude -22° 54' 10'' (-22.90278) e longitude -43° 12' 27''(-43.2075). Deverá ser criado um dataframe com as mensagens lidas;

In [3]:
# Para responder a letra a, começo baixando o pacote necessário para puxar tweets. No meu caso, usarei o 'twitteR'.
# install.packages("twitteR", repos='http://cran.us.r-project.org', lib="C:/RPackages")
# Obs.: linha comentada para não dar problema em outras máquinas

also installing the dependencies 'bit', 'curl', 'openssl', 'R6', 'bit64', 'rjson', 'DBI', 'httr'



package 'bit' successfully unpacked and MD5 sums checked
package 'curl' successfully unpacked and MD5 sums checked
package 'openssl' successfully unpacked and MD5 sums checked
package 'R6' successfully unpacked and MD5 sums checked
package 'bit64' successfully unpacked and MD5 sums checked
package 'rjson' successfully unpacked and MD5 sums checked
package 'DBI' successfully unpacked and MD5 sums checked
package 'httr' successfully unpacked and MD5 sums checked
package 'twitteR' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\NinoFabrizio\AppData\Local\Temp\Rtmp0o1gns\downloaded_packages


In [28]:
# Carregando a bibloteca
library("twitteR")

In [29]:
# Configurando meu ambiente usando as informações da minha conta no Twitter para poder puxar as informações do servidor deles
consumer_key <- 'AubI9DoHBxvW2231RiNecpTmi'
consumer_secret <- 'o0rUHjAd33MLHfTo6oPz1HjtOVd6404xEqB5pJfPY0DmNfaNem'
access_token <- '801120223504072704-MPjKnjOTCcqocYsdJfnzbPe2OwAwajN'
access_secret <- 'ORoPdE19zSyCvDUzmpiokVB7On8vcERevcKawXeNtC5hI'

setup_twitter_oauth(consumer_key, consumer_secret, access_token, access_secret)

[1] "Using direct authentication"


In [213]:
# Puxando os dados referentes ao requerido no enunciado, limitando a um número de 10000 tweets
r_stats <- searchTwitter("Olimpíada", n=10000, since='2016-09-17', geocode='-22.90278,-43.2075,60km')

# Obs.: A linha acima costuma dar problema. Mesmo assim, se persistir em dar erro, recomendo tentar a linha de baixo
# r_stats <- searchTwitter("Olimpíada", n=1000, since='2016-09-17', until='2016-11-22', geocode='-22.90278,-43.2075,60km')

# Obs.: Inicialmente eu ia excluir retweets, mas isso diminuia demais meu número total de mensagens. Abaixo deixo o código que
# mostra o número gerado ao excluir retweets
no_retweets <- strip_retweets(r_stats)
length(no_retweets)

Warning message:
In doRppAPICall("search/tweets", n, params = params, retryOnRateLimit = retryOnRateLimit, : 10000 tweets were requested but the API can only return 1394

[1] 244

In [214]:
# Verifico quantos resultados finais obtive e guardo o valor
numTweets <- length(r_stats)
numTweets

# Passo as mensagens para uma nova lista
msg <- lapply(r_stats, function(t) t$getText())

[1] 1394

In [215]:
# Agora passo as mensagens para um data frame. Anteriormente vi o tamanho da minha lista sem tweets. Esse mesmo número equivale
# ao total de tuplas no meu data frame.
msgDataFrame <- data.frame(matrix(unlist(msg), nrow=numTweets, byrow=T), stringsAsFactors=FALSE)

# Renomeio a coluna do data frame
names(msgDataFrame) <- c("Tweets")

In [216]:
# Verificando início e fim do meu data frame
head(msgDataFrame)
tail(msgDataFrame)

,Tweets
1,RT @Brittana25: Priscila e outra pessoinha que me fez gostar de escrever e acho que pela Priscila vou tentar na próxima olimpíada!
2,A única coisa que valeu a pena em 2016 foi a olimpíada
3,"É aquela coisa... Quer fazer tudo ao mesmo tempo, não consegue produzir nada no prazo certo. Esse pós-Olimpíada foi MUITO maluco."
4,RT @jairbolsonaro: Parabéns Colégio Militar de Juiz de Fora pelo 2° lugar na Olimpíada Mundial do Conhecimento. Um exemplo p/ as escolas pú…
5,Comecei a odiar a olimpíada quando descobri q ela ferrou com o cronograma inteiro da minha faculdade
6,RT @jairbolsonaro: Parabéns Colégio Militar de Juiz de Fora pelo 2° lugar na Olimpíada Mundial do Conhecimento. Um exemplo p/ as escolas pú…


,Tweets
1389,Ir a uma Olimpíada sempre foi um desejo...mas aí fui a uma Olimpíada… https://t.co/s8IIkulbbP
1390,"Pois é, até uma criança sabia em 2015 que o Rio estava quebrado. Mas tinha uma Olimpíada em 2016, né? https://t.co/GLLsT7gEYl"
1391,Posso dizer q clube já fez até estudo p/ o Engenhão de: - rebaixamento do gramado - arquibancada atrás dos gols como na Olimpíada definitiva
1392,"Nós tanto falamos dessa merda dessa Olimpíada, que ia dar ruim, mas foi só chegar o dia da abertura e pronto! O... https://t.co/WHwLevM4Tf"
1393,https://t.co/xmfv7sujl9 https://t.co/cbzCaXY9T4
1394,Mauro Ventura: 3 vezes medalhista de ouro na Olimpíada de Matemática ensina como tornar a disciplina mais atraente… https://t.co/O7EdTstldy


b) A partir dos textos lidos no item a: realizar a limpeza removendo caracteres como “.-‘´:”, pontuação, números e URLs; converter os caracteres de todas as palavras para minúsculo; remover stopwords; remover espaços;

In [217]:
# Para responder a letra b, começo por baixar o pacote 'tm' para tirar stop words dos tweets
# install.packages("tm", repos='http://cran.us.r-project.org', lib="C:/RPackages")
# Obs.: linha comentada para não dar problema em outras máquinas

In [218]:
library("tm")

In [219]:
# Criando meu grupo de stopwords em portugês do Brasil
stopwordsPTBR <- c("a", "agora", "ainda", "alguem", "algum", "alguma", "algumas", "alguns", "ampla", "amplas", "amplo", "amplos",
                 "ante", "antes", "ao", "aos", "apos", "aquela", "aquelas", "aquele", "aqueles", "aquilo", "as", "ate",
                 "atraves", "cada", "coisa", "coisas", "com", "como", "contra", "contudo", "da", "daquele", "daqueles", "das",
                 "de", "dela", "delas", "dele", "deles", "depois", "dessa", "dessas", "desse", "desses", "desta", "destas",
                 "deste", "deste", "destes", "deve", "devem", "devendo", "dever", "devera", "deverao", "deveria", "deveriam",
                 "devia", "deviam", "disse", "disso", "disto", "dito", "diz", "dizem", "do", "dos", "e", "eh", "ela", "elas",
                 "ele", "eles", "em", "enquanto", "entre", "era", "essa", "essas", "esse", "esses", "esta", "esta", "estamos",
                 "estao", "estas", "estava", "estavam", "estavamos", "este", "estes", "estou", "eu", "fazendo", "fazer", "feita",
                 "feitas", "feito", "feitos", "foi", "for", "foram", "fosse", "fossem", "grande", "grandes", "ha", "isso", "isto",
                 "ja", "la", "lhe", "lhes", "lo", "mas", "me", "mesma", "mesmas", "mesmo", "mesmos", "meu", "meus", "minha",
                 "minhas", "muita", "muitas", "muito", "muitos", "na", "nao", "nas", "nem", "nenhum", "nessa", "nessas", "nesta",
                 "nestas", "ninguem", "no", "nos", "nossa", "nossas", "nosso", "nossos", "num", "numa", "nunca", "o", "os", "ou",
                 "outra", "outras", "outro", "outros", "para", "pela", "pelas", "pelo", "pelos", "pequena", "pequenas", "pequeno",
                 "pequenos", "per", "perante", "pode", "pude", "podendo", "poder", "poderia", "poderiam", "podia", "podiam",
                 "pois", "por", "porem", "porque", "posso", "pouca", "poucas", "pouco", "poucos", "primeiro", "primeiros",
                 "propria", "proprias", "proprio", "proprios", "quais", "qual", "quando", "quanto", "quantos", "que", "quem",
                 "sao", "se", "seja", "sejam", "sem", "sempre", "sendo", "sera", "serao", "seu", "seus", "si", "sido", "so",
                 "sob", "sobre", "sua", "suas", "talvez", "tambem", "tampouco", "te", "tem", "tendo", "tenha", "ter", "teu",
                 "teus", "ti", "tido", "tinha", "tinham", "toda", "todas", "todavia", "todo", "todos", "tu", "tua", "tuas",
                 "tudo", "ultima", "ultimas", "ultimo", "ultimos", "um", "uma", "umas", "uns", "vendo", "ver", "vez", "vindo",
                 "vir", "voce", "voces", "vc", "vcs", "vos", "mais", "menos", "sim", "nao", "vai", "vao", "ser", "ngm", "toa",
                  "ir", "indo", "ia", "dar", "ai", "ali")

In [220]:
# Crio um novo data frame com meu conteúdo tratado
treatedDataFrame <- msgDataFrame

# Para cada tupla, faço as mudanças necessárias
for(i in 1:numTweets) {
    
    # Aqui se tiram URLs
    treatedDataFrame$Tweets[i] <- gsub(" ?(f|ht)(tp)(s?)(://)(.*)[.|/](.*)", " ",treatedDataFrame$Tweets[i])

    # Aqui se tiram ascentos
    treatedDataFrame$Tweets[i] <- iconv(treatedDataFrame$Tweets[[i]], from="UTF-8",to="ASCII//TRANSLIT")

    # Aqui se substituem maiúsculas por minúsculas
    treatedDataFrame$Tweets[i] <- tolower(treatedDataFrame$Tweets[i])

    # Aqui se tiram demais caracteres não desejados
    treatedDataFrame$Tweets[i] <- gsub("[][?+!#$%()*,.:;<=>@^_`^´¨¬|~.{}¦…\"\'\\\n\t/0123456789-]"," ",treatedDataFrame$Tweets[i])
    
    # Aqui tiro as stopwords
    treatedDataFrame$Tweets[i] <- removeWords(treatedDataFrame$Tweets[i][[1]],stopwordsPTBR)

    # Aqui se tiram os espaços no início e no fim
    treatedDataFrame$Tweets[i] <- lapply(treatedDataFrame$Tweets[i], function(t) gsub("^\\s+|\\s+$", "", t))
}

# Verifico início e fim da meu novo data frame
head(treatedDataFrame)
tail(treatedDataFrame)

,Tweets
1,rt brittana priscila pessoinha fez gostar escrever acho priscila vou tentar proxima olimpiada
2,unica valeu pena olimpiada
3,quer tempo consegue produzir nada prazo certo pos olimpiada maluco
4,rt jairbolsonaro parabens colegio militar juiz fora lugar olimpiada mundial conhecimento exemplo p escolas pu
5,NA
6,rt jairbolsonaro parabens colegio militar juiz fora lugar olimpiada mundial conhecimento exemplo p escolas pu


,Tweets
1389,olimpiada desejo fui olimpiada
1390,crianca sabia rio quebrado olimpiada ne
1391,dizer q clube fez estudo p engenhao rebaixamento gramado arquibancada atras gols olimpiada definitiva
1392,tanto falamos merda olimpiada ruim chegar dia abertura pronto
1393,
1394,mauro ventura vezes medalhista ouro olimpiada matematica ensina tornar disciplina atraente


In [221]:
# Vejo que alguns valores foram perdidos no meio da meu tratamento, optei por tirar as tuplas com valores NA e com valores vazios

# Retirando valores NA
tempList <- treatedDataFrame[!is.na(treatedDataFrame[,1]),]

# Retirando string vazias
tempList <- tempList[tempList != ""]

# Verificando quantos valores sobraram
listNum <- length(tempList)
listNum

# Criando o data frame final
tweetsDataFrame <- data.frame(matrix(unlist(tempList), nrow=listNum, byrow=T), stringsAsFactors=FALSE)
names(tweetsDataFrame) <- c("Tweets")

[1] 1370

In [222]:
# Verifico início e fim da meu novo data frame
head(tweetsDataFrame)
tail(tweetsDataFrame)

,Tweets
1,rt brittana priscila pessoinha fez gostar escrever acho priscila vou tentar proxima olimpiada
2,unica valeu pena olimpiada
3,quer tempo consegue produzir nada prazo certo pos olimpiada maluco
4,rt jairbolsonaro parabens colegio militar juiz fora lugar olimpiada mundial conhecimento exemplo p escolas pu
5,rt jairbolsonaro parabens colegio militar juiz fora lugar olimpiada mundial conhecimento exemplo p escolas pu
6,tive levar blusa olimpiada costureira pq tava horrivel


,Tweets
1365,capa vejario desafios barra tijuca olimpiada
1366,olimpiada desejo fui olimpiada
1367,crianca sabia rio quebrado olimpiada ne
1368,dizer q clube fez estudo p engenhao rebaixamento gramado arquibancada atras gols olimpiada definitiva
1369,tanto falamos merda olimpiada ruim chegar dia abertura pronto
1370,mauro ventura vezes medalhista ouro olimpiada matematica ensina tornar disciplina atraente


c) Realizar o stemming dos textos;

In [223]:
# Para responder a letra c, usarei o pacote 'tm' baixado anteriormente por mim.
# Mas antes, preciso baixar um novo pacote, 'SnowmallC'
# install.packages("SnowballC", repos='http://cran.us.r-project.org', lib="C:/RPackages")library(RWeka)
# Obs.: linha comentada para não dar problema em outras máquinas

In [224]:
library("SnowballC")

In [225]:
# Começo por criar um corpus contendo os dados do meu data frame
src <- DataframeSource(tweetsDataFrame)
options(mc.cores=1)
c <- Corpus(src)

# Crio uma cópia desse corpus para usar de dicionário para completar as palavras derivadas do meu stemming
c_copy <- c

In [229]:
# Faço o stemming das palavras para cada mensagem pega do data frame usando a devida função do pacote 'tm'
corpus.temp <- tm_map(c, stemDocument, language = "portuguese")

# Verifico o resultado do stemming
inspect(corpus.temp)

<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 1370

[[1]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 79

[[2]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 19

[[3]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 54

[[4]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 86

[[5]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 86

[[6]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 42

[[7]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 18

[[8]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 26

[[9]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 86

[[10]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 86

[[11]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 86

[[12]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 86

[[13]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 86

[[14]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 86

[[15]]
<<PlainTextDocument>>


In [230]:
# Finalmente, completo meu primeiro stemming às suas formas originais e verifico meu resultado final
# Obs.: A linha abaixo demora MUITO para executar
corpus.final <- tm_map(corpus.temp, stemCompletion, dictionary = c_copy)
inspect(corpus.final)

Warning message:
In grep(sprintf("^%s", w), dictionary, value = TRUE): argument 'pattern' has length > 1 and only the first element will be usedWarning message:
In grep(sprintf("^%s", w), dictionary, value = TRUE): argument 'pattern' has length > 1 and only the first element will be usedWarning message:
In grep(sprintf("^%s", w), dictionary, value = TRUE): argument 'pattern' has length > 1 and only the first element will be usedWarning message:
In grep(sprintf("^%s", w), dictionary, value = TRUE): argument 'pattern' has length > 1 and only the first element will be usedWarning message:
In grep(sprintf("^%s", w), dictionary, value = TRUE): argument 'pattern' has length > 1 and only the first element will be usedWarning message:
In grep(sprintf("^%s", w), dictionary, value = TRUE): argument 'pattern' has length > 1 and only the first element will be usedWarning message:
In grep(sprintf("^%s", w), dictionary, value = TRUE): argument 'pattern' has length > 1 and only the first element will

<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 1370

[[1]]
rt brittan priscil pessoinh fez gost escrev acho priscil vou tent proxim olimpi 
                                                                        content 
                                                                           <NA> 
                                                                           meta 

[[2]]
unic val pen olimpi                <NA> 
            content                meta 

[[3]]
quer temp conseg produz nad praz cert pos olimpi maluc 
                                               content 
                                                  <NA> 
                                                  meta 

[[4]]
rt jairbolsonar parabens colegi milit juiz lug olimpi mundial conhec exempl p escol pu 
                                                                               content 
                                                                     

d) Criar e exibir a DTM. Apresentar os termos mais encontrados e as respectivas freqüências;

In [231]:
src <- DataframeSource(tweetsDataFrame)
options(mc.cores=1)
c_copy <- c <- Corpus(src)
c <- tm_map(c, content_transformer(tolower), mc.cores=1)
c <- tm_map(c,content_transformer(removeNumbers), mc.cores=1)
c <- tm_map(c,removeWords, stopwords("portuguese"), mc.cores=1)
c <- tm_map(c,content_transformer(stripWhitespace), mc.cores=1)

#make DTM
dtm <- DocumentTermMatrix(c, control = list(tokenize = BigramTokenizer,weighting=weightTf))
freqs <- as.data.frame(inspect(dtm))
colSums(freqs)

<<DocumentTermMatrix (documents: 1370, terms: 1679)>>
Non-/sparse entries: 17474/2282756
Sparsity           : 99%
Maximal term length: 38
Weighting          : term frequency (tf)

      Terms
Docs   &gt &gt &gt estadaoesporte &gt premiacao &lt leia aberta proxima
  1          0                  0             0        0              0
  2          0                  0             0        0              0
  3          0                  0             0        0              0
  4          0                  0             0        0              0
  5          0                  0             0        0              0
  6          0                  0             0        0              0
  7          0                  0             0        0              0
  8          0                  0             0        0              0
  9          0                  0             0        0              0
  10         0                  0             0        0              0
  11         0  

&gt &gt                     &gt estadaoesporte 
                                     1                                      1 
                         &gt premiacao                               &lt leia 
                                     1                                      1 
                        aberta proxima                     abertura olimpiada 
                                     1                                      3 
                     abertura primeira                        abertura pronto 
                                     1                                      1 
                          absurda odio                       acaba esquecendo 
                                     1                                      1 
                          acabar jogos                        acabou engenhao 
                                     1                                      1 
                        acabou receber                        aceitaram texto 
                                     1                                      1 
                       aceitou assumiu                            achar video 
                                     1                                      1 
                            acho ficar                           acho momento 
                                     1                                      1 
                         acho priscila                           acho prisoes 
                                     2                                     94 
                         acho publicar                                 acho q 
                                    21                                      1 
                 acompanhando abertura                   acompanhou olimpiada 
                                     1                                      3 
                       acontece paises                        aconteceu tanta 
                                     1                                      1 
                       acordao simples                       acordo seguraram 
                                     1                                      1 
                      acreditava certo                            adaptacao p 
                                     2                                      2 
                       adorei conhecer                        agencia fiocruz 
                                     1                                      2 
                                agr to                               ah gente 
                                     1                                      2 
                               ah mano                           ah olimpiada 
                                     1                                      1 
                              aham kkk                        aimarginal pais 
                                     1                                      4 
                           album fotos                      alegria olimpiada 
                                     2                                      4 
                             alem vide                          alemanha hora 
                                     3                                      1 
                   alinnefanelli igual                            alto partic 
                                     1                                      1 
                          alturas copa                           aluno victor 
                                     1                                     17 
                        alunos colegio                              alunos es 
                                     1                                      1 
                        alunos gustavo                         alunos merecem 
                                     1                                      1 
                          amanha manha                           amanha rolar 
  

e) Apresentar um histograma (termo x freqüência) para os temos que apareceram pelo menos 40 vezes;

f) Apresentar os termos associados com o termo ouro segundo uma correlação de 60%;

g) Apresentar um wordcloud com termos cuja freqüência seja de pelo menos 20;

h) Apresentar um wordcloud com cores;

i) Apresentar o dendogram do cluster hierárquico identificando os clusters;

j) Usando o método k-means apresentar o cluster de termos;